# Create ISCIII Awards from Portal FIS

Creates Instituto de Salud Carlos III (ISCIII) awards from Portal FIS data. ~8.8K Spanish health research projects.

**Prerequisites:**
- Run `scripts/local/isciii_to_s3.py` to download and upload the data first.

**Data source:** Portal FIS (https://portalfis.isciii.es/)  
**S3 location:** `s3a://openalex-ingest/awards/isciii/isciii_projects.parquet`

**ISCIII funder:**
- funder_id: 4320334923
- ROR: https://ror.org/00ca2c886
- DOI: 10.13039/501100004587
- display_name: "Instituto de Salud Carlos III"

**Project Types:**
- PI: Proyectos de Investigación (Research Projects) - most common
- AC: Acciones (Actions)
- FI: Contratos de investigadores (Research Contracts)
- CP: Contratos Posdoctorales (Postdoctoral Contracts)
- RD: CIBER/REDES (Networks)
- COV: COVID-related projects

## Step 1: Create Staging Table from S3

In [0]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.isciii_projects_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/isciii/isciii_projects.parquet`;

In [0]:
%sql
-- Check row count (should be ~8.8K)
SELECT COUNT(*) as total_projects FROM openalex.awards.isciii_projects_raw;

In [0]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.isciii_projects_raw LIMIT 5;

## Step 2: Create ISCIII Awards Table

In [0]:
%sql
CREATE OR REPLACE TABLE openalex.awards.isciii_awards
USING delta
AS
WITH
-- Get ISCIII funder from OpenAlex by explicit funder_id
isciii_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320334923  -- Instituto de Salud Carlos III
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000 as id,

        -- Display name = project title
        g.title as display_name,

        -- Description from abstract
        g.abstract as description,

        -- Funder info
        f.funder_id,
        g.project_id as funder_award_id,

        -- Funding amount (EUR)
        g.amount as amount,
        COALESCE(g.currency, 'EUR') as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map ISCIII project codes to types
        CASE
            WHEN g.project_id LIKE 'FI%' THEN 'fellowship'
            WHEN g.project_id LIKE 'CP%' THEN 'fellowship'
            WHEN g.project_id LIKE 'RD%' THEN 'center'
            WHEN g.project_id LIKE 'DTS%' THEN 'infrastructure'
            WHEN g.project_id LIKE 'PI%' THEN 'grant'
            WHEN g.project_id LIKE 'AC%' THEN 'grant'
            WHEN g.project_id LIKE 'COV%' THEN 'grant'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme
        g.funder_scheme,

        -- Provenance
        'portal_fis_isciii' as provenance,

        -- Dates (stored as strings in YYYY-MM-DD format)
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')) as start_year,
        YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator from Portal FIS data
        CASE WHEN g.pi_family_name IS NOT NULL THEN
            struct(
                g.pi_given_name as given_name,
                g.pi_family_name as family_name,
                CAST(NULL AS STRING) as orcid,
                CAST(NULL AS DATE) as role_start,
                struct(
                    g.institution as name,
                    'ES' as country,
                    CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                ) as affiliation
            )
        ELSE NULL END as lead_investigator,

        -- Co-lead and other investigators (not available)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL
        g.landing_page_url,

        -- No DOI for Portal FIS projects
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.isciii_projects_raw g
    CROSS JOIN isciii_funder f
    WHERE g.project_id IS NOT NULL
      AND TRIM(g.project_id) != ''
)

SELECT * FROM awards_transformed;

In [0]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'portal_fis_isciii' AND priority = 33;

-- Insert into openalex_awards_raw with priority 33
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    33 as priority
FROM openalex.awards.isciii_awards;

## Verification Queries

In [0]:
%sql
-- Check row count (should be ~8.8K)
SELECT COUNT(*) as total_isciii_awards FROM openalex.awards.isciii_awards;

In [0]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    amount,
    currency,
    funding_type,
    start_date,
    end_date,
    lead_investigator.family_name as pi_name
FROM openalex.awards.isciii_awards 
LIMIT 10;

In [0]:
%sql
-- Check funder distribution (should all be ISCIII)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.isciii_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [0]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.isciii_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [0]:
%sql
-- Check project type distribution (by prefix)
SELECT 
    REGEXP_EXTRACT(funder_award_id, '^([A-Z]+)', 1) as project_type,
    COUNT(*) as cnt,
    ROUND(SUM(amount), 0) as total_funding_eur
FROM openalex.awards.isciii_awards
GROUP BY REGEXP_EXTRACT(funder_award_id, '^([A-Z]+)', 1)
ORDER BY cnt DESC;

In [0]:
%sql
-- Check data completeness and total funding
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_pi,
    ROUND(SUM(amount), 0) as total_funding_eur,
    ROUND(AVG(amount), 0) as avg_funding_eur,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_pi
FROM openalex.awards.isciii_awards;

In [0]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt, ROUND(SUM(amount), 0) as total_funding
FROM openalex.awards.isciii_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 15;